## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-15-01-00-00 +0000,wsj,"President Trump, by meeting Vladimir Putin on ...",https://www.wsj.com/politics/national-security...
1,2025-08-15-01-00-00 +0000,wsj,The nine law firms that promised President Tru...,https://www.wsj.com/politics/policy/trump-pro-...
2,2025-08-15-01-00-00 +0000,wsj,Can the U.S. Still Be Europe’s Peacemaker?,https://www.wsj.com/world/europe/can-the-u-s-s...
3,2025-08-15-01-00-00 +0000,wsj,"America’s defining mobility has stalled, leavi...",https://www.wsj.com/economy/american-job-housi...
4,2025-08-15-01-00-00 +0000,wsj,Ukrainians Who Fled War Are Losing Their Legal...,https://www.wsj.com/us-news/ukrainians-who-fle...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2317/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
1,trump,60
4,putin,24
75,will,11
82,up,11
319,year,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
0,2025-08-15-01-00-00 +0000,wsj,"President Trump, by meeting Vladimir Putin on ...",https://www.wsj.com/politics/national-security...,138
184,2025-08-14-15-59-01 +0000,nypost,Trump reveals the odds of Putin summit ending ...,https://nypost.com/2025/08/14/us-news/trump-gi...,125
231,2025-08-14-11-58-04 +0000,nypost,Russia appears ready to test new missile — eve...,https://nypost.com/2025/08/14/world-news/russi...,122
143,2025-08-14-17-56-02 +0000,nypost,India shops the globe for oil after 50% tariff...,https://nypost.com/2025/08/14/world-news/india...,115
126,2025-08-14-19-03-21 +0000,cbc,"Zelenkyy, Starmer meet ahead of Trump, Putin s...",https://www.cbc.ca/player/play/9.6867280?cmp=rss,115


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
0,138,2025-08-15-01-00-00 +0000,wsj,"President Trump, by meeting Vladimir Putin on ...",https://www.wsj.com/politics/national-security...
30,46,2025-08-14-23-11-41 +0000,startribune,"New details, charges emerge as Vance Boelter i...",https://www.startribune.com/hennepin-county-gr...
39,43,2025-08-14-22-55-34 +0000,nypost,Long Island mom smashes 14-year-old girl in th...,https://nypost.com/2025/08/14/us-news/long-isl...
302,41,2025-08-14-03-00-00 +0000,wsj,As other Asian countries ramp up manufacturing...,https://www.wsj.com/economy/trade/trump-tariff...
136,41,2025-08-14-18-15-15 +0000,nypost,Air Canada warns it will cancel 500 flights by...,https://nypost.com/2025/08/14/business/air-can...
54,36,2025-08-14-22-26-00 +0000,wsj,California Democrats are planning their own re...,https://www.wsj.com/politics/policy/gavin-news...
184,36,2025-08-14-15-59-01 +0000,nypost,Trump reveals the odds of Putin summit ending ...,https://nypost.com/2025/08/14/us-news/trump-gi...
166,34,2025-08-14-16-29-11 +0000,nypost,DC cops allowed to alert ICE about illegal imm...,https://nypost.com/2025/08/14/us-news/dc-cops-...
44,33,2025-08-14-22-48-00 +0000,nypost,Nike co-founder Phil Knight and wife pledge re...,https://nypost.com/2025/08/14/us-news/nike-co-...
262,30,2025-08-14-09-39-38 +0000,bbc,Watch: Passengers escape car hit by drone in R...,https://www.bbc.com/news/videos/cwypkvpdz3eo?a...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
